# Analyizing Jobs

This notebook will hold all analytics related code on this dataset. There are a list of questions I would like to answer and provide meaningfull visualization of the answers. Some questions are: 

 - How many jobs are posted last month, last 3 months for big tech hubs like London, Amsterdam, Auston or San Francisco? 
 - Same for countries
 - What are the best paying jobs for a given city? 
 - What are the best paying technologies for a given city?
 - What are the best paying technologies globally? 
 

In [194]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from unidecode import unidecode
import time 

from bokeh.charts import Bar, output_file, show
from bokeh.sampledata.autompg import autompg as df
from bokeh.io import output_notebook, show 

output_notebook()

jobs = pd.read_csv('../data/stackoverflow_jobs_enhanced.csv', thousands=',')
technologies = pd.read_csv('../data/technologies.csv')

# this is needed for excel export 
jobs.country = jobs.country.astype(str)
jobs.city = jobs.city.astype(str)

jobs['city']=jobs['city'].apply( lambda x:  unidecode(unicode(x, encoding = "utf-8")))  
jobs['country']=jobs['country'].apply( lambda x:  unidecode(unicode(x, encoding = "utf-8")))

Loading BokehJS ...

## Top cities and countries posting jobs 

In [195]:
top_cities = jobs.groupby(['city'])['jobid'].count().sort_values(ascending=False)
top_cities = top_cities.nlargest(15)

# create a data frame out of the series. 
# found this trick at stackoverflow: 
# http://stackoverflow.com/questions/10373660/converting-a-pandas-groupby-object-to-dataframe
df_cities = pd.DataFrame({'count' : top_cities}).reset_index()

In [196]:
p = Bar(df_cities, values='count', title='Jobs posted by citites', legend='top_right', label=CatAttr(columns=['city'], sort=False),)

# Need to add different color for different cities 
show(p)


In [201]:
top_countries = jobs.groupby(['country'])['jobid'].count().sort_values(ascending=False)
top_countries = top_countries.nlargest(15)

df_countries = top_countries.to_frame('count').reset_index()
top_countries

country
US             9830
Germany        4093
UK             2334
Netherlands    1226
Canada          702
nan             591
Australia       410
Sweden          345
Switzerland     339
France          328
Ireland         249
Spain           239
Finland         202
Austria         198
India           157
Name: jobid, dtype: int64

In [202]:
bar_countries = Bar(df_countries, values='count', title='Jobs posted by countries', legend='top_right', label=CatAttr(columns=['country'], sort=False),)

# Need to add different color for different cities 
show(bar_countries)

## Top technologies for a given city (London, Amsterdam and San Francisco)


In [204]:
# London top technologies 
london_tech = technologies[technologies.city == 'London'].groupby(['city', 'tech'])['jobid'].count().sort_values(ascending=False)
london_tech.nlargest(10)

city    tech               
London  javascript             294
        java                   276
        python                 195
        amazon-web-services    146
        c#                     136
        angularjs              106
        linux                  100
        php                     96
        tdd                     86
        html                    86
Name: jobid, dtype: int64

In [205]:
# Amsterdam top technologies 
amsterdam_tech = technologies[technologies.city == 'Amsterdam'].groupby(['city', 'tech'])['jobid'].count().sort_values(ascending=False)
amsterdam_tech.nlargest(10)

city       tech               
Amsterdam  java                   122
           javascript             106
           php                     63
           python                  59
           mysql                   48
           c++                     39
           css                     37
           c#                      37
           html                    35
           amazon-web-services     34
Name: jobid, dtype: int64

In [206]:
# Berlin 
berlin_tech = technologies[technologies.city == 'Berlin'].groupby(['city', 'tech'])['jobid'].count().sort_values(ascending=False)
berlin_tech.nlargest(10)

city    tech      
Berlin  java          364
        javascript    333
        php           174
        python        153
        mysql         152
        angularjs     125
        sql           118
        css           115
        linux          98
        html           97
Name: jobid, dtype: int64

In [207]:
# Silicon Valley 
cal_tech = technologies[technologies.state == 'CA'].groupby('tech')['jobid'].count().sort_values(ascending=False)
cal_tech.nlargest(10)

tech
javascript             542
java                   537
python                 531
c++                    282
linux                  237
amazon-web-services    213
sql                    198
angularjs              177
c#                     167
ruby-on-rails          165
Name: jobid, dtype: int64

In [166]:
# Ordered view of what technologies are the most sought after in wich city 
all_tech = technologies.groupby(['city', 'tech'])['jobid'].count().sort_values(ascending=False)
amsterdam_tech.nlargest(100)

city                tech               
New York            javascript             385
Berlin              java                   359
                    javascript             329
New York            python                 294
London              javascript             290
                    java                   273
New York            java                   227
San Francisco       python                 215
                    javascript             209
London              python                 194
San Francisco       java                   176
Berlin              php                    173
Seattle             java                   165
München             java                   158
Berlin              mysql                  152
                    python                 152
London              amazon-web-services    145
München             javascript             137
Rotterdam           php                    135
London              c#                     135
Toronto             

## Dumping out data to csv

This is only need to find values for cleaning up and normalizing values 

In [26]:
cities.to_frame('city').to_csv('../data/cities.csv', encoding = 'utf-8')
countries.to_frame('countries').to_csv('../data/countries.csv', encoding = 'utf-8')

# Writing it to Excel


In [168]:
ew = pd.ExcelWriter('../data/stackjobs.xlsx',options={'encoding':'utf-8'})
df_cities.to_excel(ew, 'City')
df_countries.to_excel(ew, 'Country')
ew.save()